In [3]:
import numpy as np
from numpy import log,sum
import os
from scipy.io.wavfile import write,read

In [4]:
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

import bokeh.plotting.figure as bk_figure
from bokeh.io import curdoc, show
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput, Button, RadioButtonGroup, Dropdown, Toggle
from bokeh.io import output_notebook
from bokeh.events import MouseLeave, Tap, DoubleTap, MouseMove

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

output_notebook()

Loading BokehJS ...

In [5]:
class Synth(object):

    def __init__(self, Fe=16000):

        object.__init__(self)
        
        self.Fe = Fe
        self._dx = 1/Fe

        
    def pure(self,A=.8,F=440,duration=3,shape='sine',random_phase=False,phase=0):

        dx = self._dx

        if (random_phase): phi = np.random.uniform(0,1)
        else: phi = phase

        x= np.arange(0,duration,self._dx)*F+phi

        if (shape=='sine'):
            res = A*np.sin(x*2*np.pi)
        elif (shape=='sawtooth'):
            x1 = x-np.floor(x)
            res = A*(2*x1-1)
        elif (shape=="triangle"):
            x1 = x-np.floor(x)
            res = A*((x1<=1./2)*(4*x1-1)+(x1>1./2)*(-4*x1+3))
        elif (shape == 'square'):
            res = A*np.sign(np.sin(2*np.pi*x))

        return(res)

    def noise(self,duration=3,sigma=.3):
        x = np.arange(0,duration,self._dx)
        n = len(x)
        y = np.random.normal(scale=sigma,size=n)
        return(y)
    
    def chord(self,chord,A=.8,duration=3,shape='sine',random_phase=False):

        res = self.pure(A=0,F=1,duration=duration)

        first = True
        for note in chord:
            A1 = 1
            p = 0
            if (random_phase): p = np.random.uniform(0,1)
            else: p=0
            f = note['f']
            a = note['a']
            on = note['on']
            if on:
                if f<0:
                    s=self.noise(duration=duration,sigma=A)
                else:
                    s = self.pure(A=a,F=f,duration=duration,shape=shape,random_phase=False,phase=p)
                if (first):
                    res = s
                    first = False
                else: res += s

        res /= len(chord)

        return res

In [6]:
class Plot(object):
    def __init__(self, sig , Fe=16000, x_display_range=None, flag_display=True):
        object.__init__(self)

        self.Fe = Fe
        self._dx = 1/Fe
        self._x = self._x_orig = np.arange(0,len(sig)*self._dx,self._dx)
        self._y = self._y_orig = sig
        self._x_min = 0
        self._x_max = self._x_orig[-1]
        self._cut_sig(x_display_range)

        self.build_display()
        
        if flag_display:
            self.display()
            

    def play(self,flag_norm=True,blocking=False,selection=True):

        if selection:
            y = self._y
        else:
            y = self._y_orig
            
        if (flag_norm): y = np.int16(y/np.max(np.abs(y)) * 32767)
        else: y = np.int16(y * 32767)

        write('test.wav', self.Fe, y)

        if (blocking):
            os.system("./play test.wav")
        else:
            os.system("./play test.wav &")
            
    def _cut_sig(self,x_range=None):
        if x_range is None:
            self._x_min = 0
            self._x_max = self._x_orig[-1]
        else:
            self._x_min,self._x_max = x_range
        n_min = int(self._x_min/self._dx)
        n_max = int(self._x_max/self._dx)
        N = n_max-n_min
        self._x = np.linspace(self._x_min,self._x_max,N)
        self._y = self._y_orig[n_min:n_max]

    def _redraw(self):
        self._source.data = dict(x=self._x, y=self._y)

    def _add_xy_widgets(self,width = 80):
        self._w_x_text = TextInput(title="",width=80,value='time')
        self._w_y_text = TextInput(title="",width=80,value='value')
        def leave(event):
            self._w_x_text.value='time'
            self._w_y_text.value='value'
        def move(event):
            self._w_x_text.value=str(event.x)
            self._w_y_text.value=str(event.y)
        self._plot.on_event(MouseLeave,leave)
        self._plot.on_event(MouseMove,move)
        return [self._w_x_text,self._w_y_text]
    
    def _add_zoom(self,width = 80):
        def zoom_left(event):
            self._cut_sig([event.x,self._x_max])
            self._redraw()                        
        def zoom_right(event):
            self._cut_sig([self._x_min,event.x])
            self._redraw()
        self._plot.on_event(Tap,zoom_left)
        self._plot.on_event(DoubleTap,zoom_right)
        
        self._w_reset_button = Button(label="Unzoom",width=80)
        self._w_reset_button.background='green'
        def unzoom(event):
            self._cut_sig()
            self._redraw()            
        self._w_reset_button.on_click(unzoom)
        
        return(self._w_reset_button)

    def _add_play(self,width = 80):
        def play_selection(event):
            self.play()
        def play_all(event):
            self.play(selection=False)
        
        self._w_play_selection_button = Button(label="Play",width=80)
        self._w_play_selection_button.background='green'
        self._w_play_selection_button.on_click(play_selection)
        
        self._w_play_all_button = Button(label="PlayA",width=80)
        self._w_play_all_button.background='green'
        self._w_play_all_button.on_click(play_all)

        return([self._w_play_selection_button,self._w_play_all_button])
    
    def set_sig(self, sig, flag_redraw=True, flag_play_all=True):
        self._x_orig = np.arange(0,len(sig)*self._dx,self._dx)
        self._y_orig = sig
        self._cut_sig(x_range=[self._x_min,self._x_max])
        if flag_redraw:
            self._redraw()
        if flag_play_all:
            self.play(selection=False)
        
    def build_display(self):        
        
        self._source = ColumnDataSource(data=dict(x=self._x, y=self._y))
        self._plot = bk_figure(plot_height=500, plot_width=500, title="", tools="")
        self._plot.line('x', 'y', source=self._source, line_width=1, line_alpha=0.6)
        
        wunzoom = self._add_zoom()
        wx,wy = self._add_xy_widgets()
        wp,wpa = self._add_play()
        
        self.layout = widgetbox(self._plot,row(wx,wy,wunzoom,wp,wpa))
        
    def display(self):
        
        def modify_doc(doc):
            doc.add_root(row(self.layout, width=800))
            doc.title = "Sliders"
        handler = FunctionHandler(modify_doc)
        app = Application(handler)
        show(app) 

In [7]:
class HarmonicWidget(object):

    def __init__(self, Fe= 16000, A=.8, F=261.63, duration=1, x_display_range=[0,.02], n_harmonics = 12):
        
        object.__init__(self)
        
        self._Fe = Fe
        self._synth = Synth(Fe)
        
        self._A = A
        
        self._n_harmonics = n_harmonics
        self._shape_labels = ["sine","triangle","sawtooth","square","noise"]
        
        self._duration = duration
        self._random_phase = False
        
        self._factor = 1
        
        self._synth = Synth(Fe)
        sig = self._synth.pure(A=A, F=F, duration=duration, shape='sine', random_phase=False, phase=0)
        
        self._plot = Plot(sig, Fe=Fe, x_display_range=x_display_range, flag_display=False)

        w_F1,w_F,w_harmonic = self._add_harmonic_widget(F)
        w_shape = self._add_shape_widget()
        
        w_op,w_om,w_fp,w_fm = self._add_transpose_widget()
        
        c = widgetbox(row(w_F1,w_F),w_harmonic,w_shape,row(w_op,w_om,w_fp,w_fm))
        layout = row(self._plot.layout,c)
        
        def modify_doc(doc):
            doc.add_root(row(layout, width=800))
            doc.title = "Sliders"
            
        handler = FunctionHandler(modify_doc)
        app = Application(handler)
        
        show(app) 

    def _update(self):
        n = self._w_harmonics_radiogroup.active  
        shape = self._shape_labels[self._w_shape_radiogroup.active]
        self._w_F_cur.value = str(float(self._w_F1.value)*(n+1)*self._factor)
        F = float(self._w_F_cur.value)
        if shape=='noise':
            sig = self._synth.noise(duration=self._duration,sigma=.3)
        else:
            sig = self._synth.pure(A=self._A, F=F, duration=self._duration, 
                                   shape=shape, random_phase=self._random_phase, 
                                   phase=0)
        self._plot.set_sig(sig)
       
    def _add_harmonic_widget(self,F):
        def update_widget1(event):
            self._factor = 1
            self._update()
        def update_widget3(a,b,c):
            self._factor = 1
            self._update()
        self._w_harmonics_radiogroup = RadioButtonGroup(labels=["1","2","3","4","5","6","7","8","9","10","11","12"], active=0)
        self._w_harmonics_radiogroup.on_click(update_widget1)        
        self._w_F1 = TextInput(title="F1",width=80,value=str(F))
        self._w_F_cur = TextInput(title="F",width=80,value=str(F))
        self._w_F1.on_change('value',update_widget3)        
        
        return([self._w_F1,self._w_F_cur,self._w_harmonics_radiogroup]) 
        
    def _add_shape_widget(self):
        shape_labels = ["sine","triangle","sawtooth","square","noise"]
        self._w_shape_radiogroup = RadioButtonGroup(labels=shape_labels, active=0)
        def update_widget(event):
            self._update()
        self._w_shape_radiogroup.on_click(update_widget)
        return(self._w_shape_radiogroup)
        
    def _add_transpose_widget(self):
        def updateop(v):
            self._factor *= 2
            self._update()
        def updateom(v):
            self._factor /= 2
            self._update()
        def updatefp(v):
            self._factor *= 3/2
            self._update()
        def updatefm(v):
            self._factor /= 3/2
            self._update()
        self._w_op = Button(label="Octave+",width=80)
        self._w_om = Button(label="Octave-",width=80)
        self._w_fp = Button(label="Fifth+",width=80)
        self._w_fm = Button(label="Fith-",width=80)
        self._w_op.on_click(updateop)
        self._w_om.on_click(updateom)
        self._w_fp.on_click(updatefp)
        self._w_fm.on_click(updatefm)

        return([self._w_op,self._w_om,self._w_fp,self._w_fm])

In [8]:
class ChordWidget(object):

    def __init__(self, notes=None, Fe= 16000, A=.8, duration=1, x_display_range=[0,.02]):
        
        object.__init__(self)
        
        self._Fe = Fe
        self._synth = Synth(Fe)
        
        self._A = A
        
        self._shape_labels = ["sine","triangle"]
        
        self._duration = duration
        self._random_phase = False
        
        self._synth = Synth(Fe)

        if notes:
            self._n_harmonics = len(notes)
            nn = []
            for n in range(self._n_harmonics):
                nn.append(dict(f=notes[n][0],a=notes[n][1],on=notes[n][2]==1))
            notes = nn
        else:
            self._n_harmonics = 5
            notes = []
            for n in range(self._n_harmonics):
                notes.append(dict(f=440,a=1,on=False)) 
            notes[0]['on']=True
            
        sig = self._synth.chord(chord = notes,duration=duration)  
        self.sig = sig
        
        self._plot = Plot(sig, Fe=Fe, x_display_range=x_display_range, flag_display=False)

        widgets = []

        def update_widget1_(v1):
            notes = []
            for n in range(self._n_harmonics):
                notes.append(dict(f=440,a=1,on=False)) 
            for n in range(self._n_harmonics):
                notes[n]['f'] = float(widgets[n]['f'].value)
                notes[n]['a'] = float(widgets[n]['a'].value)
                notes[n]['on'] = widgets[n]['on'].active
            random_phase = random_phase_toggle.active
            shape = shape_labels[shape_radiogroup.active]
            y = self._synth.chord(chord = notes,duration=duration,random_phase=random_phase,shape=shape)  
            self._plot.set_sig(y)

        def update_widget1(v1,v2,v3):
            update_widget1_(v1)

        widgetcolumns = [[],[],[]]
        for n in range(self._n_harmonics):
            freq = TextInput(value=str(notes[n]['f']),title="f"+str(n+1),width=80)
            amp = TextInput(value=str(notes[n]['a']),title="a"+str(n+1),width=80)
            on = Toggle(label="On/Off "+str(n+1),width=80,active=notes[n]['on'])
            d = dict(f=freq,a=amp,on=on)
            widgets.append(d) 
            widgetcolumns[0].append(freq)
            widgetcolumns[1].append(amp)
            widgetcolumns[2].append(on)
            amp.on_change('value',update_widget1)
            freq.on_change('value',update_widget1)
            on.on_click(update_widget1_)
            
        shape_labels = ["sine","triangle"]
        shape_radiogroup = RadioButtonGroup(labels=shape_labels, active=0)        
        shape_radiogroup.on_click(update_widget1_)

        random_phase_toggle = Toggle(label="RPhase",width=80,active=False)
        random_phase_toggle.on_click(update_widget1_)
        widgetcolumns[1].append(random_phase_toggle)
        widgetcolumns[2].append(shape_radiogroup)

        # Set up layouts and add to document
        layout = row(self._plot.layout,widgetbox(widgetcolumns[0]),widgetbox(widgetcolumns[1]),widgetbox(widgetcolumns[2]))

        def modify_doc(doc):
            doc.add_root(row(layout, width=800))
            doc.title = "Sliders"
            
        handler = FunctionHandler(modify_doc)
        app = Application(handler)
        show(app) 

In [7]:
# Clarinet 
Fe,s = read("clarinette.wav")
Plot(s,Fe=Fe)

In [13]:
# Chat 
Fe,s = read("chat.wav")
Plot(s,Fe=Fe)

In [33]:
# I.1 Two building blocks : the pure sound and the white noise
ChordWidget(notes=[(440,.8,1),(-1,1.3,0)])

/opt/miniconda3/envs/py3k/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [29]:
# I.2 Pure sound 
HarmonicWidget()

In [34]:
# I.4 Masking 
ChordWidget(notes=((440,.1,1),(-1,1.3,1)))

In [49]:
# I.4 : interferences
ChordWidget(notes = [(440,1,1),(429.29998,1,1)],duration=5,x_display_range=[0,1])

In [50]:
# I.4 : interferences
ChordWidget(notes = [(440,1,1),(429.29998,1,1),(440.5,1,1),(439.14999,1,1)],duration=5,x_display_range=[0,1])

In [51]:
# I.4 : interferences
ChordWidget(notes = [(440,1,1),(443,1,1),(442.0999,1,1),(437,1,1),(439,1,1)],duration=5,x_display_range=[0,1])

In [54]:
# I.4 Interférences
r=os.system('afplay risset-interf.aiff')

/opt/miniconda3/envs/py3k/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [40]:
# I.4 The pythagorician comma
r =531441/524288
F1 = 520
ChordWidget(notes=[(F1,1,1),(F1*r,1,1)],duration=3)

In [41]:
# The pythagorician perfect chord versus the harmonic one (5/4)
F1 = 260
ChordWidget(notes=[(F1,1,1),(F1*81/64,1,1),(F1*5/4,1,0),(F1*3/2,1,1)],duration=3)

In [42]:
# Exemple de transpositions délirantes !?!

In [52]:
# I.4 Pitch is periodicity
ChordWidget(notes=[(440,.7,0),(1320,1,1),(2200,1,1),(2640,1,1),(3080,1,1)])

/opt/miniconda3/envs/py3k/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [44]:
# I.4 What about timber ? Chords versus monophonic 
ChordWidget(notes=[(220,1,1),(440,.5,1),(660,.3,1),(880,0.2,1)])

In [45]:
# More fun 

In [57]:
r=os.system('afplay risset-timbre.aiff')

In [59]:
r=os.system('afplay risset-transpo.aiff')

In [60]:
r=os.system('afplay risset-acceler.aiff')

In [7]:
# Trumpet 
Fe,s = read("clarinette.wav")
s = s/np.max(np.abs(s))
Plot(s,Fe=Fe)

In [8]:
nb_bits = 8
nq = 2**nb_bits/2
ns = np.round(s*nq)/nq
Plot(ns-s,Fe=Fe)

In [10]:
Plot(ns,Fe=Fe)

In [14]:
s = ChordWidget(notes=[(220,1,1),(440,.5,1),(660,.3,1),(880,0.2,1)])

In [16]:
s = s.sig/np.max(np.abs(s.sig))
nb_bits = 8
nq = 2**nb_bits/2
ns = np.round(s*nq)/nq
Plot(ns-s,Fe=Fe)

## ARMA

In [9]:
def arma(duration,r,F,Fe=16000):
    N = duration*Fe
    a1 = -2*r*np.cos(2*np.pi*F/Fe)
    a2 = r*r
    input = np.zeros(N+2)
    input[0] = 0
    input[1] = 1
    output = np.zeros(N+2)
    for n in range(N):
        output[n+2] = -a1*output[n+1]-a2*output[n]+input[n]
    return(output)
  

In [10]:
u = arma(2,.9998,440)
Plot(u)

In [12]:
u = arma(2,.9993,440)
Plot(u)

In [11]:
u = arma(2,.9988,440)
Plot(u)

In [65]:
Fe,s = read("Ircam-cht-piano_wood-interp.wav")
Plot(s,Fe=Fe)


In [ ]:
Fe,s = read("Ircam-cht-cloche-deform.wav")
Plot(s,Fe=Fe)

## Quantification

In [114]:
# X : Son pur de petite amplitude
A = .0003
synth = Synth()
x = synth.pure(duration=2,A=A)
Plot(x,x_display_range=[0,0.04])

In [109]:
# Q(X) : Quantifié sur 16 bits
qx = np.round(x*(2**15))/2**15
Plot(u,x_display_range=[0,0.04])

In [110]:
# X-Q(X) : L'erreur de quantification
Plot(x-qx,x_display_range=[0,0.04])

In [111]:
# X+W : On rajoute un bruit blanc
xw = s+np.random.normal(0,A/40, len(s))
Plot(xw,x_display_range=[0,0.04])

In [112]:
# Q(X+W) Quantifié sur 16 bits
qxw = np.round(xw*(2**15))/2**15
Plot(qxw,x_display_range=[0,0.04])

In [113]:
# X+W - Q(X+W)
Plot(xw-qxw,x_display_range=[0,0.04])